In [50]:
import os
import pandas as pd
import platform
import selenium
import pickle

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

In [51]:
operating_system = platform.system()

if operating_system == 'Linux':
    webdriver_path = os.path.join('chromedriver_linux64', 'chromedriver')
elif operating_system == 'Windows':
    webdriver_path = os.path.join('chromedriver_win32', 'chromedriver.exe')

service = Service(executable_path = webdriver_path)

In [52]:
def get_artists_names(elements):
    artists_names = []
    
    for element in elements:
        artists_names.append(element.text.strip())
    
    return artists_names

In [53]:
# Para salvar o index 

arquivo_numero = "numero.txt"

def salvar_numero(numero):
    with open(arquivo_numero, "w") as arquivo:
        arquivo.write(str(numero))

def carregar_numero():
    try:
        with open(arquivo_numero, "r") as arquivo:
            return int(arquivo.read())
    except FileNotFoundError:
        return None  

In [55]:
# Para salvar a lista com nomes dos artistas

arquivo_lista = "artists_names.pkl"

def salvar_lista(lista):
    with open(arquivo_lista, "wb") as arquivo:
        pickle.dump(lista, arquivo)

def carregar_lista():
    try:
        with open(arquivo_lista, "rb") as arquivo:
            return pickle.load(arquivo)
    except FileNotFoundError:
        return None

In [56]:
driver = webdriver.Chrome(service = service)
web_driver_wait = WebDriverWait(driver, 30)

In [74]:
artists_names = carregar_lista()
index_inicial = carregar_numero()

for i in range (index_inicial, 478):
    link = f'https://www.last.fm/tag/brazilian/artists?page={i}'
    driver.get(link)
    
    artist_name_identifier = 'big-artist-list-title'
    elements = web_driver_wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, artist_name_identifier)))
    current_artists_names = get_artists_names(elements)
    artists_names += current_artists_names
    
    salvar_lista(artists_names)
    salvar_numero(i + 1)

In [79]:
dict_ = {'artist': artists_names}
df = pd.DataFrame.from_dict(dict_)
df.to_csv('artists_names.csv')